<a href="https://colab.research.google.com/github/jayasharm-a/Covid-vs-Pneumonia/blob/main/Covid_vs_Pneumonia_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load COVID dataset
covid_df = pd.read_csv('../Dataset/Covid/metadata.csv')
#covid_df['label'] = 'COVID'  # Add a new column to label the data as COVID




In [ ]:
covid_df.head()

,directory,class
0,covid/COVID19(308).jpg,1
1,covid/COVID-19 (839).jpg,1
2,covid/COVID-19 (105).jpg,1
3,covid/COVID-19 (379).jpg,1
4,covid/COVID19(443).jpg,1


In [ ]:
covid_df.shape

(4575, 2)

In [ ]:
covid_df

,directory,class
0,covid/COVID19(308).jpg,1
1,covid/COVID-19 (839).jpg,1
2,covid/COVID-19 (105).jpg,1
3,covid/COVID-19 (379).jpg,1
4,covid/COVID19(443).jpg,1
...,...,...
4570,normal/00000398_004.png,0
4571,normal/00000975_000.png,0
4572,normal/00000766_009.png,0
4573,normal/00000105_003.png,0


In [ ]:
df_y = covid_df['class']
df_x = covid_df['directory']

In [ ]:
df_x

0         covid/COVID19(308).jpg
1       covid/COVID-19 (839).jpg
2       covid/COVID-19 (105).jpg
3       covid/COVID-19 (379).jpg
4         covid/COVID19(443).jpg
                  ...           
4570     normal/00000398_004.png
4571     normal/00000975_000.png
4572     normal/00000766_009.png
4573     normal/00000105_003.png
4574     normal/00000697_001.png
Name: directory, Length: 4575, dtype: object

In [ ]:
df_y

0       1
1       1
2       1
3       1
4       1
       ..
4570    0
4571    0
4572    0
4573    0
4574    0
Name: class, Length: 4575, dtype: int64

In [ ]:
df_y.value_counts()

1    1525
2    1525
0    1525
Name: class, dtype: int64

In [ ]:

import cv2
import os
from datetime import datetime
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df_x, df_y, stratify=df_y, test_size=0.20, random_state=7)

<IPython.core.display.Javascript object>

In [ ]:
Y_train.value_counts()

0    1220
1    1220
2    1220
Name: class, dtype: int64

In [ ]:
Y_test.value_counts()

1    305
2    305
0    305
Name: class, dtype: int64

In [ ]:
test = pd.concat([X_test, Y_test], axis = 1)
test.head()

,directory,class
614,covid/COVID-19 (506).jpg,1
2473,pneumonia/person23_bacteria_82.jpeg,2
501,covid/COVID19(241).jpg,1
912,covid/COVID-19 (219).jpg,1
1270,covid/COVID-19 (549).jpg,1


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
from tensorflow.keras.layers import Lambda, Reshape, DepthwiseConv2D, ZeroPadding2D, Add, MaxPooling2D,Activation, Flatten, Conv2D, Dense, Input, Dropout, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import to_categorical
import gc
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, Activation, BatchNormalization, Dropout, LSTM, ConvLSTM2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint,TensorBoard,TerminateOnNaN, LearningRateScheduler
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.layers import Input,Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation, LSTM, ConvLSTM2D, Lambda, Reshape, BatchNormalization, Bidirectional
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import shutil

In [ ]:
X_train[:10]

3502                    normal/00000242_003.png
3855                    normal/00000798_002.png
529                       covid/COVID19(16).jpg
3660                    normal/00001068_003.png
4371                    normal/00001039_004.png
2698    pneumonia/person1040_bacteria_2974.jpeg
991                    covid/COVID-19 (636).jpg
1499                   covid/COVID-19 (786).jpg
3743                    normal/00000538_000.png
1679     pneumonia/person423_bacteria_1858.jpeg
Name: directory, dtype: object

In [ ]:
def filter_images():
    input_path="../Dataset/Covid/COVID19_Pneumonia_Normal_Chest_Xray_PA_Dataset"
    output_path="../Dataset/output/covid19-pneumonia-normal-chest-xray-pa-dataset"
    classs=['normal','covid','pneumonia']
    for c in classs:
        if not os.path.exists(output_path  + c):
            os.makedirs(output_path  + c)
    for row in X_train:
        path_from = "{}/{}".format(input_path, row)
        path_to = "{}/{}".format(output_path, row)
        if os.path.exists(path_from):
            shutil.copy(path_from,path_to)
filter_images()

In [ ]:
# used to copy files according to each fold
def copy_images(covid_df, directory):

    # input and output directory
    input_path = "../Dataset/Covid/COVID19_Pneumonia_Normal_Chest_Xray_PA_Dataset"
    output_path = "../Dataset/" + directory

    # remove all files from previous fold
    if os.path.exists(output_path):
        shutil.rmtree(output_path)

    # create folder for files from this fold
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # create subfolders for each class
    classs=['normal','covid','pneumonia']
    for c in classs:
        if not os.path.exists(output_path + c):
            os.makedirs(output_path + c)

  # copy files
    for row in covid_df:
        path_from = "{}/{}".format(input_path, row)
        path_to = "{}/{}".format(output_path, row)
        if os.path.exists(path_from):
            shutil.copy(path_from, path_to)
        #print(path_from)

In [ ]:
copy_images(test,'test')

In [ ]:
!dir "../dataset/test"

 Volume in drive D is DATA
 Volume Serial Number is 567E-1F8B

 Directory of D:\Machine_Learning\major project\Prject-HealthCare\dataset\test

02-03-2023  00:19    <DIR>          .
02-03-2023  00:19    <DIR>          ..
02-03-2023  00:19    <DIR>          covid
02-03-2023  00:19    <DIR>          normal
02-03-2023  00:19    <DIR>          pneumonia
               0 File(s)              0 bytes
               5 Dir(s)  642,144,776,192 bytes free


In [ ]:
len(os.listdir('../dataset/test/covid'))

305

In [ ]:
len(os.listdir('../dataset/test/normal'))

305

In [ ]:
len(os.listdir('../dataset/test/pneumonia'))

124

In [ ]:
len(os.listdir('../Dataset/Covid/COVID19_Pneumonia_Normal_Chest_Xray_PA_Dataset/pneumonia/'))

1525

In [ ]:
def get_model():


    img_dims=224
    inputs = Input(shape=(img_dims, img_dims, 3))


    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', trainable=False, padding='same')(inputs)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', trainable=False, padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)


    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', trainable=False, padding='same')(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', trainable=False, padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=(2, 2))(x)

    x = Flatten(name="flatten")(x)
    x = Dense(units=64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    output = Dense(units=3, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=output)
    opt = RMSprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

    return model

In [ ]:
from math import floor
from sklearn.model_selection import StratifiedKFold
N_FOLDS = 5
EPOCHS = 50
INIT_LR = 3e-4
T_BS = 16
V_BS = 16
decay_rate = 0.95
decay_step = 1

skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True)
log_dir = "./logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks = [ModelCheckpoint(filepath='best_cnn_model.h5', monitor='val_loss',mode='min',verbose=1,save_best_only=True,save_weights_only=True),
             LearningRateScheduler(lambda epoch : INIT_LR * pow(decay_rate, floor(epoch / decay_step))), tensorboard_callback]

In [ ]:
class_to_label_map = {2 : 'pneumonia', 1 : 'covid', 0 : 'normal'}

In [ ]:
submission_prediction = []
submission_predictions = []

for epoch, (train_index, val_index) in enumerate(skf.split(X_train, Y_train)):
    x_train, x_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train, y_val = Y_train.iloc[train_index], Y_train.iloc[val_index]

    print(y_train.value_counts())
    print(y_val.value_counts())

    train = pd.concat([x_train, y_train], axis=1)
    val = pd.concat([x_val, y_val], axis = 1)

    # copy the images according to the fold
    copy_images(train, 'train')
    copy_images(val, 'val')

    print('Running fold '+ str(epoch+1))

    # CREATE MODEL
    model = get_model()
    model.summary()

0    976
2    976
1    976
Name: class, dtype: int64
1    244
2    244
0    244
Name: class, dtype: int64
Running fold 1


D:\Anaconda\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
batch_normalization_3 (Batch (None, 224, 224, 64)      256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 112, 112, 128)     1475

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
batch_normalization_15 (Batc (None, 224, 224, 64)      256       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 112, 112, 128)     1475

In [ ]:
train.reset_index(inplace=True)
train.index = train['index']
train.drop(['level_0','index'],axis=1,inplace=True)

In [ ]:
train

,directory,class
index,,
3502,normal/00000242_003.png,0
3855,normal/00000798_002.png,0
529,covid/COVID19(16).jpg,1
3660,normal/00001068_003.png,0
4371,normal/00001039_004.png,0
...,...,...
951,covid/COVID19(225).jpg,1
3374,normal/00000373_000.png,0
681,covid/COVID-19 (43).jpg,1


In [ ]:
model.fit(
  train,train,
  batch_size=len(folders),
  validation_data=(test_X,test_y),
  epochs=50,
  shuffle=True,
  callbacks=Callbacks
  )